In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
sys.path.append('/content/drive/MyDrive/#fastcampus')
drive_project_root = '/content/drive/MyDrive/#fastcampus'
!pip install -r '/content/drive/MyDrive/#fastcampus/requirements.txt'

In [ ]:
from datetime import datetime

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from omegaconf import OmegaConf, DictConfig    # DictConfig is for time checking

import hydra
from hydra.core.config_store import ConfigStore

import tensorflow as tf
import tensorflow_probability as tfp  # 직접 구현해도 되지만 요즘은 이런 게 잘 되어 있으니 편리하게 여기서 가져다 사용하는 편
import tensorflow_addons as tfa

import wandb

In [ ]:
from config_utils_tf import flatten_dict, register_config, get_optimizer_element, get_callbacks

## make model

VAE
- generative model은 학습이 느린 편
- GPU 3개 썼는데 2주가 넘어야 학습되기도 함

In [ ]:
class VAE(tf.keras.Model):
    # convolutional variational autoencoder 모델을 사용할 예정
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        self.latent_dim = cfg.model.latent_dim  # auto encoder에서 잠재변수 차원은 중요 -> 따로 정의
        self.encoder = tf.keras.Sequential(
            [
             # 일단 2개 정도만 쌓을 예정
             # 너무 많이 쌓으면 하루가 지나도 결과가 안 나옴;;
             # Conv2D : 자체 activation 제공하고 있기 때문에 따로 쓸 필요 없음
             tf.keras.layers.Conv2D(**cfg.model.enc.conv1),
             tf.keras.layers.Conv2D(**cfg.model.enc.conv2),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(cfg.model.enc.out_fc.units),
            ],
            name="encoder",
        ) 
        self.decoder = tf.keras.Sequential(
            [
             # encoder의 Dense layer를 convolutional 2d transpose에 맞는 형태로 바꾸기
             tf.keras.layers.Dense(
                 units=cfg.model.dec.in_fc.units,
                 activation=tf.nn.relu
             ),
             tf.keras.layers.Reshape(
                 target_shape=tuple(cfg.model.dec.reshape_shape)
             ),
             # Transpose convolution 사용 (= deep convolution)
             tf.keras.layers.Conv2DTranspose(**cfg.model.dec.tr_conv1),
             tf.keras.layers.Conv2DTranspose(**cfg.model.dec.tr_conv2),
             tf.keras.layers.Conv2DTranspose(**cfg.model.dec.tr_conv3),
            ],
            name="decoder",
        )

        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.recon_loss_tracker = tf.keras.metrics.Mean(name="recon_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    # compile된 metric이 아닌 customize해서 사용하기 위함
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.recon_loss_tracker,
            self.kl_loss_tracker,
        ]

    # call, train_step, test_step은 tensorflow2(keras)에서 기본적으로 제공하는 것
    # 따라서 compile이 자동으로 됨
    # but, 그 외 'sample' 같은 건 안 됨
    # -> @tf.function 이라는 decorator 붙여서 compile 되도록 써야 함
    # encode, reparameterize, decode 등에 @tf.function 안 다는 이유는
    # call에서 불러서 사용할 함수이기 때문 (call에서 사용되는 경우 자동 compile됨)

    # sample : 임의의 값을 생성하는 generator 모델의 목적
    @tf.function
    def sample(self, epsilon=None, sample_size=100):
        if epsilon is None:
            epsilon = tf.random.normal(shape=(sample_size, self.latent_dim))
        return self.decode(epsilon)

    def encode(self, x, training=False):
        # tf.split : encoder에서 받아 온 module을 2개로 쪼갬
        mu, logvar = tf.split(
            self.encoder(x, training=training),
            num_or_size_splits=2,
            axis=1
        )
        return mu, logvar
    
    # logvar : sigmoid
    def reparameterize(self, mu, logvar):
        # epsilon의 random 값을 가지고 gaussian (normal) distribution에서 임의로 가져옴
        epsilon = tf.random.normal(shape=mu.shape)
        return mu + epsilon * tf.exp(logvar * .5)   # z

    def decode(self, z, training=False):
        return tf.sigmoid(self.decoder(z, training=training))

    def call(self, input, training=False):
        mu, logvar = self.encode(input, training=training)
        z = self.reparameterize(mu, logvar)
        output = self.decode(z, training=training)
        return output, z, mu, logvar

    def train_step(self, data):
        images, _ = data
        # images = [batch * 28 * 28]
        # 지금 convolutional layer를 쓰고 있기 때문에 images = [batch * 28 * 28 * 1]이 되어야 함
        images = tf.cast(tf.expand_dims(images, -1), tf.float32)

        with tf.GradientTape() as tape:            
            outputs, z, z_mu, z_logvar = self(images, training=True)

            # calculate loss
            # VAE의 loss는 recons loss 와 kld loss의 합으로 이루어짐

            # reconstruction loss
            # reduce mean & reduce sum : image 별로, batch 별로 따로 계산하기 위함
            recon_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.mae(images, outputs),
                    axis=(1, 2)  # image size : width, height
                )
            )

            # kld_loss
            kl_loss = -0.5 + (1 + z_logvar - tf.square(z_mu) - tf.exp(z_logvar))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

            # total_loss
            # 원하면 kl_loss에 대해 가중치 줘도 됨
            total_loss = recon_loss + kl_loss
        
        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(total_loss, trainable_vars)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # update the metrics
        # 내가 원하는 게 tensorflow에서 제공되지 않는 metrics이기 때문에 customize 하여 metrics 사용
        self.total_loss_tracker.update_state(total_loss)
        self.recon_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        # tensorboard image update
        # 이 이미지를 wandb랑 sync 하는 코드를 항상 넣어놓았으니
        # 이 이미지 업데이트 되면 wandb에서도 찍힘
        # max_outputs=5 : batch가 32개 들어간 다음에 맨 앞에 있는 5개만 사용하겠다 -> 너무 많아지면 느려지니까
        tf.summary.image("train_source_img", images, max_outputs=5)  # tensorflow log에 저장
        tf.summary.image("train_recon_img", outputs, max_outputs=5)

        # return a dict mapping metrics names to current values
        # metric에 total loss가 이미 있기 때문에 그대로 사용
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        images, _ = data
        # images = [batch * 28 * 28]
        # 지금 convolutional layer를 쓰고 있기 때문에 images = [batch * 28 * 28 * 1]이 되어야 함
        # images = tf.expand_dims(images, -1).astype(tf.float32)  # -> 오류 발생함 : 아래와 같이 바꾸기
        images = tf.cast(tf.expand_dims(images, -1), tf.float32)
       
        outputs, z, z_mu, z_logvar = self(images, training=False)

        # calculate loss
        # VAE의 loss는 recons loss 와 kld loss의 합으로 이루어짐

        # reconstruction loss
        # reduce mean & reduce sum : image 별로, batch 별로 따로 계산하기 위함
        recon_loss = tf.reduce_mean(
            tf.reduce_sum(
                tf.keras.losses.mae(images, outputs),
                axis=(1, 2)  # image size : width, height
            )
        )

        # kld_loss
        kl_loss = -0.5 + (1 + z_logvar - tf.square(z_mu) - tf.exp(z_logvar))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        # total_loss
        # 원하면 kl_loss에 대해 가중치 줘도 됨
        total_loss = recon_loss + kl_loss

        # update the metrics
        # 내가 원하는 게 tensorflow에서 제공되지 않는 metrics이기 때문에 customize 하여 metrics 사용
        self.total_loss_tracker.update_state(total_loss)
        self.recon_loss_tracker.update_state(recon_loss)
        self.kl_loss_tracker.update_state(kl_loss)

        # tensorboard image update
        # 이 이미지를 wandb랑 sync 하는 코드를 항상 넣어놓았으니
        # 이 이미지 업데이트 되면 wandb에서도 찍힘
        # max_outputs=5 : batch가 32개 들어간 다음에 맨 앞에 있는 5개만 사용하겠다 -> 너무 많아지면 느려지니까
        tf.summary.image("val_source_img", images, max_outputs=5)  # tensorflow log에 저장
        tf.summary.image("val_recon_img", outputs, max_outputs=5)

        # return a dict mapping metrics names to current values
        # metric에 total loss가 이미 있기 때문에 그대로 사용
        logs = {m.name: m.result() for m in self.metrics}
        return logs

In [ ]:
class GAN(tf.keras.Model):
    # convolutional variational autoencoder 모델을 사용할 예정
    def __init__(self, cfg: DictConfig):
        super().__init__()
        self.cfg = cfg
        self.latent_dim = cfg.model.latent_dim  # auto encoder에서 잠재변수 차원은 중요 -> 따로 정의
        # GAN에서는 encoder와 비슷한 게 discriminator (image를 받아서 classification하니까)
        self.discriminator = tf.keras.Sequential(
            [
             # 일단 2개 정도만 쌓을 예정
             # 너무 많이 쌓으면 하루가 지나도 결과가 안 나옴;;
             # Conv2D : 자체 activation 제공하고 있기 때문에 따로 쓸 필요 없음
             tf.keras.layers.Conv2D(**cfg.model.dis.conv1),
             tf.keras.layers.Conv2D(**cfg.model.dis.conv2),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(1),  # 실제 이미지인지 fake인지 구분
            ],
            name="discriminator",
        ) 
        self.generator = tf.keras.Sequential(
            [
             # encoder의 Dense layer를 convolutional 2d transpose에 맞는 형태로 바꾸기
             tf.keras.layers.Dense(
                 units=cfg.model.gen.in_fc.units,
                 activation=tf.nn.relu
             ),
             tf.keras.layers.Reshape(
                 target_shape=tuple(cfg.model.gen.reshape_shape)
             ),
             # Transpose convolution 사용 (= deep convolution)
             tf.keras.layers.Conv2DTranspose(**cfg.model.gen.tr_conv1),
             tf.keras.layers.Conv2DTranspose(**cfg.model.gen.tr_conv2),
             tf.keras.layers.Conv2DTranspose(**cfg.model.gen.tr_conv3),
            ],
            name="generator",
        )

        self.generator_loss_tracker = tf.keras.metrics.Mean(name="generator_loss")
        self.discriminator_loss_tracker = tf.keras.metrics.Mean(name="discriminator_loss")
        # loss를 generator, discriminator 따로 정의를 하고
        # GradientTape를 따로 붙여야 하기 때문에 compile 못 씀
        # -> 따라서 범용으로 쓸 loss 식 정의
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.SUM
        )

    # compile된 metric이 아닌 customize해서 사용하기 위함
    @property
    def metrics(self):
        return [
            self.generator_loss_tracker,
            self.discriminator_loss_tracker,
        ]
    
    # keras에서 제공하는 compile은 optimizer가 한 개라고 가정하는데
    # GAN은 discriminator와 generator 각각에 대한 optimizer 총 2개가 있어 compile에 대해 따로 함수 생성 필요
    def compile(self, d_optimizer, g_optimizer, **kwargs):
        super().compile(**kwargs)  # 이렇게 하면 하위 호환성을 최대한 만족시키면서 원하는 것 정의 가능
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer

    @tf.function
    def get_generator_loss(self, fake_outputs):
        return self.cross_entropy(
            tf.ones_like(fake_outputs),  # label
            fake_outputs
        ) / fake_outputs.shape[0]   # batch size에 맞도록
    
    @tf.function
    def get_discriminator_loss(self, real_outputs, fake_outputs):
        real_loss = self.cross_entropy(
            tf.ones_like(real_outputs),
            real_outputs
        )
        fake_loss = self.cross_entropy(
            tf.zeros_like(fake_outputs),
            fake_outputs
        )
        total_d_loss = (real_loss + fake_loss) / (real_outputs.shape[0] + fake_outputs.shape[0])
        return total_d_loss

    # call, train_step, test_step은 tensorflow2(keras)에서 기본적으로 제공하는 것
    # 따라서 compile이 자동으로 됨
    # but, 그 외 'sample' 같은 건 안 됨
    # -> @tf.function 이라는 decorator 붙여서 compile 되도록 써야 함
    # encode, reparameterize, decode 등에 @tf.function 안 다는 이유는
    # call에서 불러서 사용할 함수이기 때문 (call에서 사용되는 경우 자동 compile됨)

    # sample : 임의의 값을 생성하는 generator 모델의 목적
    @tf.function
    def sample(self, sample_size=100):
        # generator에 맞게 shape 지정
        return tf.random.normal(shape=(sample_size, 1, 1, self.latent_dim))

    # GAN은 input을 받지 않고 무조건 sampling을 통해 학습함
    def call(self, _, training=False):
        noise = self.sample(sample_size=self.cfg.train.train_batch_size)
        generated_images = self.generator(noise, training=training)
        discriminator_results = self.discriminator(generated_images, training=training)
        return generated_images, discriminator_results

    def train_step(self, data):
        images, _ = data
        # images = [batch * 28 * 28]
        # 지금 convolutional layer를 쓰고 있기 때문에 images = [batch * 28 * 28 * 1]이 되어야 함
        images = tf.cast(tf.expand_dims(images, -1), tf.float32)

        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:            
            outputs, fake_output = self(images, training=True)  # images 대신 None 넣어도 됨
            real_output = self.discriminator(images, training=True)

            # Calculate the loss for each item in the batch
            g_loss = self.get_generator_loss(fake_output)
            d_loss = self.get_discriminator_loss(real_output, fake_output)
        
        # compute gradients
        g_gradients = g_tape.gradient(g_loss, self.generator.trainable_variables)
        d_gradients = d_tape.gradient(d_loss, self.discriminator.trainable_variables)

        # update weights
        self.g_optimizer.apply_gradients(zip(g_gradients, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(d_gradients, self.discriminator.trainable_variables))

        # update the metrics
        self.generator_loss_tracker.update_state(g_loss)
        self.discriminator_loss_tracker.update_state(d_loss)

        # tensorboard image update
        # 이 이미지를 wandb랑 sync 하는 코드를 항상 넣어놓았으니
        # 이 이미지 업데이트 되면 wandb에서도 찍힘
        # max_outputs=5 : batch가 32개 들어간 다음에 맨 앞에 있는 5개만 사용하겠다 -> 너무 많아지면 느려지니까
        tf.summary.image("train_real_img", images, max_outputs=5)  # tensorflow log에 저장
        tf.summary.image("train_generated_img", outputs, max_outputs=5)  # fake image

        # return a dict mapping metrics names to current values
        # metric에 total loss가 이미 있기 때문에 그대로 사용
        logs = {m.name: m.result() for m in self.metrics}
        return logs
    
    def test_step(self, data):
        images, _ = data
        # images = [batch * 28 * 28]
        # 지금 convolutional layer를 쓰고 있기 때문에 images = [batch * 28 * 28 * 1]이 되어야 함
        # images = tf.expand_dims(images, -1).astype(tf.float32)  # -> 오류 발생함 : 아래와 같이 바꾸기
        images = tf.cast(tf.expand_dims(images, -1), tf.float32)
       
        outputs, fake_output = self(images, training=True)  # images 대신 None 넣어도 됨
        real_output = self.discriminator(images, training=True)

        # Calculate the loss for each item in the batch
        g_loss = self.get_generator_loss(fake_output)
        d_loss = self.get_discriminator_loss(real_output, fake_output)

                # update the metrics
        self.generator_loss_tracker.update_state(g_loss)
        self.discriminator_loss_tracker.update_state(d_loss)

        # tensorboard image update
        # 이 이미지를 wandb랑 sync 하는 코드를 항상 넣어놓았으니
        # 이 이미지 업데이트 되면 wandb에서도 찍힘
        # max_outputs=5 : batch가 32개 들어간 다음에 맨 앞에 있는 5개만 사용하겠다 -> 너무 많아지면 느려지니까
        tf.summary.image("val_real_img", images, max_outputs=5)  # tensorflow log에 저장
        tf.summary.image("val_generated_img", outputs, max_outputs=5)  # fake image

        # return a dict mapping metrics names to current values
        # metric에 total loss가 이미 있기 때문에 그대로 사용
        logs = {m.name: m.result() for m in self.metrics}
        return logs

## Configuration 정의

### data configuration

In [ ]:
data_fashion_mnist_cfg: dict = {
    "n_class": 10,
    "train_val_split": [0.9, 0.1],
    "train_val_shuffle": True,
    "train_val_shuffle_buffer_size": 1024,
    "test_shuffle": False,
    "tset_shuffle_buffer_size": 1024,
}

### model configuration

In [ ]:
model_mnist_vae_cfg: dict = {
    "name": "VAE",
    "data_normalize": True,
    "latent_dim": 2,   # 잘 floting 하려면 latent_dim이 2이면 좋음
    "enc": {
        "conv1": {
            "filters": 32,
            "kernel_size": 3,
            "strides": [2, 2],
            "activation": "relu",
        },
        "conv2": {
            "filters": 64,
            "kernel_size": 3,
            "strides": [2, 2],
            "activation": "relu",

        },
        "out_fc": {
            "units": 4,  # latent_dim의 두 배를 해야 split에 대한 대응 가능 (mu, log_var)
        },
    },
    "dec": {
        "in_fc": {
            # enc에서 받아와야 함
            # test 돌려서 잘 되는 것 찾기 -> 지금은 7*7*32가 잘 됨
            "units": 7*7*32,
        },
        "reshape_shape": [7, 7, 32],
        "tr_conv1": {
            "filters": 64,
            "kernel_size": 3,
            "strides": [2, 2],
            "padding": "same",
            "activation": "relu",
        },
        "tr_conv2": {
            "filters": 32,
            "kernel_size": 3,
            "strides": [2, 2],
            "padding": "same",
            "activation": "relu",
        },
        "tr_conv3": {
            "filters": 1,
            "kernel_size": 3,
            "strides": [1, 1],
            "padding": "same",
        },
    }
}

model_mnist_gan_cfg: dict = {
    "name": "GAN",
    "data_normalize": True,
    "latent_dim": 64,   # GAN은 완전 noise에서 가져오고 어떤 도움도 안 받기 때문에 latent 차원이 좀 큰 게 좋음
    # discriminator
    "dis": {
        "conv1": {
            "filters": 32,
            "kernel_size": 3,
            "strides": [2, 2],
            "activation": "relu",
            "padding": "same",   # default : valid
        },
        "conv2": {
            "filters": 64,
            "kernel_size": 3,
            "strides": [2, 2],
            "activation": "relu",
            "padding": "same",
        },
    },
    # generator
    "gen": {
        "in_fc": {
            "units": 7*7*32,
        },
        "reshape_shape": [7, 7, 32],
        "tr_conv1": {
            "filters": 64,
            "kernel_size": 3,
            "strides": [2, 2],
            "padding": "same",
            "use_bias": False, # generator 부분을 좀 빡세게 하기 위해서 bias를 아예 안 써보려고 함
            "activation": "relu",
        },
        "tr_conv2": {
            "filters": 32,
            "kernel_size": 3,
            "strides": [2, 2],
            "padding": "same",
            "activation": "relu",
        },
        "tr_conv3": {
            "filters": 1,
            "kernel_size": 3,
            "strides": [1, 1],
            "padding": "same",
            "activation": "sigmoid",
        },
    }
}

### optimizer configuration

In [ ]:
adam_warmup_lr_sch_opt_cfg = {
    "optimizer": {
        "name": "Adam",
        "other_kwargs": {},
    },
    "lr_scheduler": {
        "name": "LinearWarmupLRSchedule",
        "kwargs": {
            "lr_peak": 1e-3,
            "warmup_end_steps": 1500,
        }
    }
}

# RAdam은 scheduler 필요 없었음
radam_no_lr_sch_opt_cfg = {
    "optimizer": {
        "name": "RectifiedAdam",
        "learning_rate": 1e-3,
        "other_kwargs": {},
    },
    "lr_scheduler": None
}

gan_opt_cfg = {
    # discriminator와 generator optimizer를 서로 다른 거 쓰는 게 좋음
    # + 둘 다 너무 좋은 거 (rectified adam) 쓰면 성능 오히려 안 좋을 수 있음
    "discriminator": {
        "optimizer": {
            "name": "RMSprop",
            "learning_rate": 0.005,
            "other_kwargs": {}
        },
        "lr_scheduler": None,
    },
    "generator": {
        "optimizer": {
            "name": "Adam",
            "learning_rate": 0.001,
            "other_kwargs": {
                "beta_1": 0.5
            },
        },
        "lr_scheduler": None,
    }
}

# train_cfg
train_cfg: dict = {
    "train_batch_size": 256,
    "val_batch_size": 32,
    "test_batch_size": 32,
    "max_epochs": 50,
    "distribute_strategy": "MirroredStrategy",   # colab(notebook)이 아니고 다른 server에서 하면 다른 strategy 필요
}

_merged_cfg_presets = {
    "vae_fashion_mnist_radam": {
        "data": data_fashion_mnist_cfg,
        "model": model_mnist_vae_cfg,
        "opt": radam_no_lr_sch_opt_cfg,
        "train": train_cfg
    },
    "gan_fashion_mnist": {
        "data": data_fashion_mnist_cfg,
        "model": model_mnist_gan_cfg,
        "opt": gan_opt_cfg,
        "train": train_cfg
    }
}

### hydra composition ###
# clear hydra instance -> Jupyter 환경에서 할 때는 일단 instance clear 하기
hydra.core.global_hydra.GlobalHydra.instance().clear()

# register preset configs
register_config(_merged_cfg_presets)

# initialization
hydra.initialize(config_path=None)    # yaml을 쓰고 있고 외부에서 하면 config_path 지정해야 함

# using_config_key = "cnn_fashion_mnist_radam"
using_config_key = "gan_fashion_mnist"
cfg = hydra.compose(using_config_key)

# define & override log _cfg
model_name = cfg.model.name
run_dirname = "fastcampus_generative_model_tutorials_tf"
run_name = f"{datetime.now().isoformat(timespec='seconds')}-{using_config_key}-{model_name}"
log_dir = os.path.join(drive_project_root, "runs", run_dirname, run_name)

log_cfg = {
    "run_name": run_name,
    "callbacks": {
        "TensorBoard": {
            "log_dir": log_dir,
            "update_freq": 10,
        },
        # generative model은 classification이 상대적으로 불안정함
        # 어느 정도로 setting해야 적당한지 감이 잘 안 오기 때문에 이 부분 없앰
        # "EarlyStopping": {
        #     "patience": 3,
        #     "verbose": True,
        # }
    },
    "wandb": {
        "project": "fastcampus_generative_model_tutorials_tf",
        "name": run_name,
        "tags": ["fastcampus_generative_model_tutorials_tf"],
        "reinit": True,
        "sync_tensorboard": True,
    }
}

# unlock struct of config & set log config
OmegaConf.set_struct(cfg, False)
cfg.log = log_cfg

# relock config
OmegaConf.set_struct(cfg, True)
print(OmegaConf.to_yaml(cfg))

# save yaml
# with open(os.path.join(log_dir, "config.yaml")) as f:
# with open("config.yaml", "w") as f:
#     OmegaConf.save(cfg, f)

# This would open the file we saved above
# and tell you the result of the model and its configs (weights, ...)
# You can check it whenever you want
# OmegaConf.load()

In [ ]:
def get_distribute_strategy(strategy_name: str, **kwargs):
    return getattr(tf.distribute, strategy_name)(**kwargs)

distribute_strategy = get_distribute_strategy(cfg.train.distribute_strategy)

In [ ]:
with distribute_strategy.scope():
    # 데이터 셋 정의 
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    
    # normalization
    if cfg.model.data_normalize:
        x_train = x_train / 255.0
        x_test = x_test / 255.0

    # train/val splits
    assert sum(cfg.data.train_val_split) == 1.0
    train_size = int(len(x_train) * cfg.data.train_val_split[0])
    val_size = len(x_train) - train_size

    dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    if cfg.data.train_val_shuffle:
        dataset = dataset.shuffle(
            buffer_size=cfg.data.train_val_shuffle_buffer_size,
        )
    if cfg.data.test_shuffle:
        test_dataset = test_dataset.shuffle(
            buffer_size=cfg.data.test_shuffle_buffer_size,
        )

    train_dataset = dataset.take(train_size)
    val_dataset = dataset.skip(train_size)
    print(len(train_dataset), len(val_dataset), len(dataset), len(test_dataset))
    
    # dataloader 정의
    train_batch_size = cfg.train.train_batch_size
    val_batch_size = cfg.train.val_batch_size
    test_batch_size = cfg.train.test_batch_size

    train_dataloader = train_dataset.batch(train_batch_size, drop_remainder=True)
    val_dataloader = val_dataset.batch(val_batch_size, drop_remainder=True)
    test_dataloader = test_dataset.batch(test_batch_size, drop_remainder=True)

sample_example = next(iter(train_dataloader))
# print(sample_example)

## define model

LinearWarmupLRScheduler 하는 이유
- SGD는 다른 optimizer 대비 learning rate 값에 매우 민감
  - learning rate를 잘 setting 해야 성능이 좋게 나옴 (Adam보다 더 좋게 나오기도 함)
- 따라서 optimizer와 함께 learning rate도 tuning 하는 게 원래는 좋음
- 그러나 학습 속도가 너무 느려지는 단점

warmup을 하기 어려운 상황이면?
- Rectified Adam으로 먼저 테스트 해 보고, optimizer는 조절해도 거의 결과 비슷하게 나오니, 모델링 부분을 업데이트 해 보기
- Rectified Adam에도 tuning 할 수 있는 요소 많음
  - https://www.tensorflow.org/addons/api_docs/python/tfa/optimizers/RectifiedAdam

In [ ]:
# 모델 정의
def get_model(cfg: DictConfig):
    if cfg.model.name == 'VAE':
        model = VAE(cfg)
    elif cfg.model.name == "GAN":
        model = GAN(cfg)
    else:
        raise NotImplementedError()
    
    return model
    
with distribute_strategy.scope():
    model = get_model(cfg)

    # define optimizer & scheduler
    if cfg.model.name == "GAN":  # compile 방식이 다르기 때문에 GAN과 그외 방식 구분 필요
        d_optimizer, d_scheduler = get_optimizer_element(
            cfg.opt.discriminator.optimizer, cfg.opt.discriminator.lr_scheduler
        )
        g_optimizer, g_scheduler = get_optimizer_element(
            cfg.opt.generator.optimizer, cfg.opt.generator.lr_scheduler
        )
        model.compile(
            d_optimizer=d_optimizer,
            g_optimizer=g_optimizer,
            # run_eagerly = True,
        )
    else:
        optimizer, scheduler = get_optimizer_element(
            cfg.opt.optimizer, cfg.opt.lr_scheduler
        )

        # model.compile(optimizer = optimizer, run_eagerly=True)  # run_eagerly=True : for debugging
        model.compile(optimizer = optimizer)  # -> 끄면 학습 빨라짐

        # model build
        # 이 부분 생략해도 되지만 build를 해 놓으면 나중에 debugging하기 좋음 -> 권장
        # batch 1 : 임의로 설정
        # model.build((1, 28*28*1))
        model.build((train_batch_size, 28, 28, 1))      # This build code is for 'CNN'

        # 만약 build 안 하고 summary 하면 build, fit을 하거나 input shape를 넣으라고 경고 뜸
        # fit은 학습이기 때문에 무거운 감이 있고 빠르게 하기 위해 build 선호
        model.summary()

## get callbacks

In [ ]:
callbacks = get_callbacks(cfg.log)

## wandb setup

- https://docs.wandb.ai/guides/integrations/tensorflow
- sync_tensorboard=True : tensorflow에 적혀있는 걸 wandb에 업로드

In [ ]:
# flatten_dict(cfg)   # 전부 flatten 하게 바꿔주는 함수 -> nested 구조를 모두 under bar 형태로 바꿈

In [ ]:
wandb.init(
    config= flatten_dict(cfg),
    **cfg.log.wandb
)

In [ ]:
# tensorboard load하기 : load extension
%load_ext tensorboard

# 경로 지정 : terminal 문법이기 때문에 #을 # 그대로 인지하려면 앞에 '\' 써줘야 함
%tensorboard --logdir /content/drive/MyDrive/\#fastcampus/runs/fastcampus_generative_model_tutorials_tf

model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=cfg.train.max_epochs,
    callbacks=callbacks,
)

## model testing

In [ ]:
# model.evaluate(test_dataloader)

In [ ]:
def get_latent_img(model, n, single_img_size=28):
    # plot n * n images decoded from the latent_space
    
    norm = tfp.distributions.Normal(0, 1)
    grid_x = norm.quantile(np.linspace(0.05, 0.95, n))  # 0.05 ~ 0.95 중 n개 이미지 가져오기
    grid_y = norm.quantile(np.linspace(0.05, 0.95, n))
    width = single_img_size * n
    height = width
    image = np.zeros((height, width))
    # print(image.shape)

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z = np.array([[xi, yi]])
            x_decoded = model.sample(z)
            digit = tf.reshape(x_decoded[0], (single_img_size, single_img_size))
            # print(digit.shape)
            image[
                  i * single_img_size: (i + 1) * single_img_size,
                  j * single_img_size: (j + 1) * single_img_size 
            ] = digit.numpy()
    return image

latent_img = get_latent_img(model, n=10)
plt.figure(figsize=(10, 10))
plt.imshow(latent_img, cmap="Greys_r")
plt.axis("Off")
plt.show()